In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
df = pd.read_csv("drive/MyDrive/Hck/sequence_esm1_t34_670M_UR100_cls_embeddings.csv")
df.head()

In [3]:
df_main = pd.read_csv("drive/MyDrive/Hck/train.csv")
df_main.head()

,ID,protein_name,sequence,biological_process,cellular_component,molecular_function
0,train_0,sp|Q9J523|V212_FOWPN,MPQNKVLSFPLPEGTLLEDITKNKWILGKQLGSGGFGLVYQVSCKS...,0,0,1
1,train_1,sp|Q14990|ODFP1_HUMAN,MAALSCLLDSVRRDIKKVDRELRQLRCIDEFSTRCLCDLYMHPYCC...,0,0,1
2,train_2,sp|Q9C994|DTX14_ARATH,MDSAEKGLLVVSDREEVNKKDGFLRETKKLSYIAGPMIAVNSSMYV...,0,0,1
3,train_3,sp|Q58425|CARA_METJA,MEAVLILEDGTILKGKGFGAEKEVFGELVFTTVMTGYVEVLTDPSY...,0,1,0
4,train_4,sp|P78814|SRP2_SCHPO,MSETRLFVGRIPPQATREDMMDFFKGYGQILDCKLMNGFGFVEVED...,0,0,1


In [7]:
joined_df = pd.merge(df, df_main, on='ID')
joined_df = joined_df.drop(columns=['protein_name', 'sequence'])
joined_df.head()

,ID,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,...,emb_1273,emb_1274,emb_1275,emb_1276,emb_1277,emb_1278,emb_1279,biological_process,cellular_component,molecular_function
0,train_0,0.081642,-1.496138,6.191256,0.679270,-1.894954,4.502281,-0.514480,3.705019,-0.492262,...,2.121812,-0.186634,2.419172,1.625475,-1.370067,3.178513,-1.555542,0,0,1
1,train_1,-0.339906,-0.648377,5.818838,-0.628044,-3.921365,4.332178,-1.524251,4.316974,-0.746856,...,3.244596,-0.824896,0.850447,-2.062417,1.170538,-2.074513,-2.184298,0,0,1
2,train_10,1.769063,-1.486243,7.261370,-1.914409,-2.150486,4.021339,-1.634238,4.790095,-0.374175,...,1.596033,0.121016,1.480196,-0.077234,-1.287797,3.832566,1.213811,0,0,1
3,train_100,-0.875047,-0.755958,6.746736,0.169595,-3.256755,5.133801,-2.103694,5.345797,0.062203,...,2.900385,0.083600,1.125013,-1.735611,1.038142,0.507925,-1.397163,1,0,0
4,train_1000,1.375432,-1.485533,6.716955,-0.113301,-3.485360,4.589830,1.116525,4.952724,1.901705,...,3.332405,-1.423786,2.461106,-0.964605,-0.446336,1.997515,-1.162120,0,0,1


In [10]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Assuming `joined_df` is your final DataFrame after preprocessing
features = joined_df.drop(columns=['ID', 'biological_process', 'cellular_component', 'molecular_function']).values
targets = joined_df[['biological_process', 'cellular_component', 'molecular_function']].values

# Convert targets to categorical (one-hot encoding) if they're not already
targets = to_categorical(targets)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)


In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

# Define the RNN model
model = Sequential()
model.add(SimpleRNN(128, input_shape=(X_train.shape[1], 1), return_sequences=True))
model.add(SimpleRNN(64))
model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))



In [14]:
history = model.fit(X_train_reshaped, y_train, epochs=10, validation_split=0.2)


Epoch 1/10
673/673 [==============================] - 581s 859ms/step - loss: 1.1035 - accuracy: 0.3629 - val_loss: 1.0930 - val_accuracy: 0.3716
Epoch 2/10
673/673 [==============================] - 573s 851ms/step - loss: 1.0967 - accuracy: 0.3708 - val_loss: 1.2346 - val_accuracy: 0.3429
Epoch 3/10
673/673 [==============================] - 566s 841ms/step - loss: 1.1045 - accuracy: 0.3355 - val_loss: 1.0997 - val_accuracy: 0.3470
Epoch 4/10
673/673 [==============================] - 567s 843ms/step - loss: 1.1009 - accuracy: 0.3388 - val_loss: 1.0995 - val_accuracy: 0.3470
Epoch 5/10
673/673 [==============================] - 567s 842ms/step - loss: 1.1013 - accuracy: 0.3420 - val_loss: 1.0996 - val_accuracy: 0.3470
Epoch 6/10
673/673 [==============================] - 561s 832ms/step - loss: 1.1019 - accuracy: 0.3404 - val_loss: 1.0991 - val_accuracy: 0.3396
Epoch 7/10
673/673 [==============================] - 561s 834ms/step - loss: 1.1016 - accuracy: 0.3434 - val_loss: 1.1013 -